# L4: Support Data Insight Analysis

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Loading Tasks and Agents YAML files

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Using FileReadTool

In [3]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support_tickets_data.csv')

## Creating Agents, Tasks and Crew

In [4]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool]
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=False
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)


## Testing our Crew

In [5]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

2025-05-14 17:09:59,531 - 140547977546624 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./support_tickets_data.csv\"}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,resolved
T0001,C0511,API Issue,I'm pleased with how my issue was handled. Thanks!,High,2023-03-25,24



# Agent: Suggestion Engine
## Final Answer: 
1. **Ticket T0001 - API Issue**  
   - Suggest reviewing past successful resolutions to improve response times for API issues.  
   - Consider providing follow-up to ensure full resolution and prevent recurrence.  

2. **Ticket T0002 - Login Issue**  
   - Suggest sending a follow-up email to check if the issue persists and if any additional support is needed.  
   - Utilize the positive feedback to train agents on effective communication and resolution strategies.  

3. **Ticket T0003 - Report Generation**  
   - Suggest reevaluating the report generation process to identify bottlenecks.  
   - Consider providing a step-by-step guide to users to facilitate ease of report generation.

4. **Ticket T0004 - Data Import**  
   - Suggest maintaining clear communication with the customer about the steps taken to resolve their issue.  
   - Encourage agents to gather additional feedback when issues are resolved to improve future service.  

5. **

# Agent: Chart Specialist
## Task: Generate charts based on the tables provided by the previous task. The charts should include: - Issue Distribution: A chart showing the distribution of different issue
  types.
- Priority Levels: A chart depicting the breakdown of tickets by priority
  level.
- Resolution Times: A trend line showing average resolution times over the
  past months.
- Customer Satisfaction: A bar chart or trend line showing customer
  satisfaction ratings over time.
- Agent Performance: A chart showing the performance of different agents
  based on resolution times and customer satisfaction scores.

Save the charts as image files in the current directory.
Ensure that the charts are saved as image files, and generate URLs for these images so they can be easily embedded into the final report.



# Agent: Chart Specialist
## Final Answer: 
**1. Issue Distribution Chart**  
This chart shows the distribution of different issue types based on their frequency.  
![Issue Distri



# Agent: Report Generator
## Final Answer: 
### Support Ticket Summary Report

#### 1. Issue Classification Results
| Issue Type        | Frequency | Priority Levels (High/Medium/Low/Critical) |
|-------------------|-----------|----------------------------------------------|
| **API Issue**         | 10        | High: 6, Medium: 3, Low: 1                  |
| **Login Issue**       | 10        | High: 3, Medium: 5, Low: 2, Critical: 1      |
| **Data Import**       | 10        | High: 4, Medium: 5, Low: 1                  |
| **Feature Request**   | 7         | High: 4, Medium: 2, Low: 1, Critical: 2    |
| **Billing Issue**     | 10        | High: 5, Medium: 3, Low: 2, Critical: 3    |
| **Report Generation**  | 7         | High: 3, Medium: 3, Low: 1                  |
| **UI Bug**            | 6         | High: 3, Medium: 2, Low: 1, Critical: 1    |

#### 2. Agent Performance
| Agent ID | Total Tickets | Average Resolution Time (minutes) | Satisfaction Rating (Average) |
|----------

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  9.0  │    9.0     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  9.0  │    9.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │ 10.0  │    10.0    │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │  9.0  │    9.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 9.25  │    9.2     │                     │  │  ┃
┃ Execution Time (s) │   0   │     0      │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

## Training your crew and agents

In [ ]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

2025-05-14 17:10:03,436 - 140547977546624 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./support_tickets_data.csv\"}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,resolved
T0001,C0511,API Issue,I'm pleased with how my issue was handled. Thanks!,High,2023-03-25,24



# Agent: Suggestion Engine
## Final Answer: 
1. **Ticket T0001 - API Issue**  
   - Suggest reviewing past successful resolutions to improve response times for API issues.  
   - Consider providing follow-up to ensure full resolution and prevent recurrence.  

2. **Ticket T0002 - Login Issue**  
   - Suggest sending a follow-up email to check if the issue persists and if any additional support is needed.  
   - Utilize the positive feedback to train agents on effective communication and resolution strategies.  

3. **Ticket T0003 - Report Generation**  
   - Suggest reevaluating the report generation process to identify bottlenecks.  
   - Consider providing a step-by-step guide to users to facilitate ease of report generation.

4. **Ticket T0004 - Data Import**  
   - Suggest maintaining clear communication with the customer about the steps taken to resolve their issue.  
   - Encourage agents to gather additional feedback when issues are resolved to improve future service.  

5. **

## Comparing new test results

In [ ]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

In [ ]:
# Display the Trello screenshot
from IPython.display import Image, display

# Load and display the image
test_image = Image(filename='test_before_training.png', width=368)
display(test_image)

## Kicking off Crew

In [ ]:
result = support_report_crew.kickoff()

## Result

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))